In [3]:
import torch
from fall_detection_data_driven import SimpleCNN
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
# 首先创建模型实例
model_action = SimpleCNN(3)  # 确保SimpleCNN构造函数调用匹配保存状态时的模型结构

# 然后加载状态字典
model_path = 'F:\SRIBD\ESP32-Realtime-System\weights.pth'
model_action.load_state_dict(torch.load(model_path, map_location=device))


<All keys matched successfully>

In [25]:

# 可修改的参数
# Reduce displayed waveforms to avoid display freezes
CSI_VAID_SUBCARRIER_INTERVAL = 1 # interval越大，有效载波数越少

# 默认参数
# Remove invalid subcarriers
# secondary channel : below, HT, 40 MHz, non STBC, v, HT-LFT: 0~63, -64~-1, 384
csi_vaid_subcarrier_index = []
csi_vaid_subcarrier_color = []
color_step = 255 // (28 // CSI_VAID_SUBCARRIER_INTERVAL + 1)

# LLTF: 52
csi_vaid_subcarrier_index += [i for i in range(6, 32, CSI_VAID_SUBCARRIER_INTERVAL)]     # 26  red
csi_vaid_subcarrier_color += [(i * color_step, 0, 0) for i in range(1,  26 // CSI_VAID_SUBCARRIER_INTERVAL + 2)]
csi_vaid_subcarrier_index += [i for i in range(33, 59, CSI_VAID_SUBCARRIER_INTERVAL)]    # 26  green
csi_vaid_subcarrier_color += [(0, i * color_step, 0) for i in range(1,  26 // CSI_VAID_SUBCARRIER_INTERVAL + 2)]
CSI_DATA_LLFT_COLUMNS = len(csi_vaid_subcarrier_index)
# HT-LFT: 56 + 56
csi_vaid_subcarrier_index += [i for i in range(66, 94, CSI_VAID_SUBCARRIER_INTERVAL)]    # 28  blue
csi_vaid_subcarrier_color += [(0, 0, i * color_step) for i in range(1,  28 // CSI_VAID_SUBCARRIER_INTERVAL + 2)]
csi_vaid_subcarrier_index += [i for i in range(95, 123, CSI_VAID_SUBCARRIER_INTERVAL)]   # 28  White
csi_vaid_subcarrier_color += [(i * color_step, i * color_step, i * color_step) for i in range(1,  28 // CSI_VAID_SUBCARRIER_INTERVAL + 2)]
# csi_vaid_subcarrier_index += [i for i in range(124, 162)]  # 28  White
# csi_vaid_subcarrier_index += [i for i in range(163, 191)]  # 28  White

CSI_DATA_COLUMNS = len(csi_vaid_subcarrier_index)

DATA_COLUMNS_NAMES = ["type", "id", "mac", "rssi", "rate", "sig_mode", "mcs", "bandwidth", "smoothing", "not_sounding", "aggregation", "stbc", "fec_coding",
                      "sgi", "noise_floor", "ampdu_cnt", "channel", "secondary_channel", "local_timestamp", "ant", "sig_len", "rx_state", "len", "first_word", "data"]



In [61]:
# Reduce displayed waveforms to avoid display freezes
CSI_VAID_SUBCARRIER_INTERVAL = 1

# Remove invalid subcarriers
# secondary channel : below, HT, 40 MHz, non STBC, v, HT-LFT: 0~63, -64~-1, 384
csi_vaid_subcarrier_index = []
csi_vaid_subcarrier_color = []
color_step = 255 // (28 // CSI_VAID_SUBCARRIER_INTERVAL + 1)

# LLTF: 52
csi_vaid_subcarrier_index += [i for i in range(6, 32, CSI_VAID_SUBCARRIER_INTERVAL)]     # 26  red
csi_vaid_subcarrier_color += [(i * color_step, 0, 0) for i in range(1,  26 // CSI_VAID_SUBCARRIER_INTERVAL + 2)]
csi_vaid_subcarrier_index += [i for i in range(33, 59, CSI_VAID_SUBCARRIER_INTERVAL)]    # 26  green
csi_vaid_subcarrier_color += [(0, i * color_step, 0) for i in range(1,  26 // CSI_VAID_SUBCARRIER_INTERVAL + 2)]
CSI_DATA_LLFT_COLUMNS = len(csi_vaid_subcarrier_index)


In [67]:

# # HT-LFT: 56 + 56
# csi_vaid_subcarrier_index += [i for i in range(66, 94, CSI_VAID_SUBCARRIER_INTERVAL)]    # 28  blue
# csi_vaid_subcarrier_color += [(0, 0, i * color_step) for i in range(1,  28 // CSI_VAID_SUBCARRIER_INTERVAL + 2)]
# csi_vaid_subcarrier_index += [i for i in range(95, 123, CSI_VAID_SUBCARRIER_INTERVAL)]   # 28  White
# csi_vaid_subcarrier_color += [(i * color_step, i * color_step, i * color_step) for i in range(1,  28 // CSI_VAID_SUBCARRIER_INTERVAL + 2)]
# # csi_vaid_subcarrier_index += [i for i in range(124, 162)]  # 28  White
# # csi_vaid_subcarrier_index += [i for i in range(163, 191)]  # 28  White

CSI_DATA_INDEX = 200  # buffer size
CSI_DATA_COLUMNS = len(csi_vaid_subcarrier_index)
DATA_COLUMNS_NAMES = ["type", "id", "mac", "rssi", "rate", "sig_mode", "mcs", "bandwidth", "smoothing", "not_sounding", "aggregation", "stbc", "fec_coding",
                      "sgi", "noise_floor", "ampdu_cnt", "channel", "secondary_channel", "local_timestamp", "ant", "sig_len", "rx_state", "len", "first_word", "data"]
# csi_data_array = np.zeros(
#     [CSI_DATA_INDEX, CSI_DATA_COLUMNS], dtype=np.complex64)
import pandas as pd
import numpy as np
import json

# 使用pandas读取CSV文件
df = pd.read_csv('F:\\SRIBD\\ESP32-Realtime-System\\fall.csv')

# 定义csi_data_array的大小，根据之前的设置
CSI_DATA_LLFT_COLUMNS = len(csi_vaid_subcarrier_index)
# CSI_DATA_LLFT_COLUMNS = 52
csi_data_array = np.zeros([len(df),CSI_DATA_LLFT_COLUMNS], dtype=np.complex64)

# 遍历DataFrame中的每一行
for index, row in df.iterrows():
    # print(index)
    csi_string = row['data']  # 假设CSV的列名为'data'，包含CSI数据的JSON字符串
    # print(csi_string)
    try:
        csi_raw_data = json.loads(csi_string)
    except json.JSONDecodeError:
        print(f"data is incomplete at row {index}")
        continue
    
    # 检查CSI数据的长度是否有效
    if len(csi_raw_data) not in [128, 256, 384]:
        print(f"element number is not equal: {len(csi_raw_data)} at row {index}")
        continue
    
    # 根据有效子载波索引提取CSI数据并填充到csi_data_array中
    for i in range(CSI_DATA_LLFT_COLUMNS):
        # print(True)
        csi_data_array[index,i] = complex(csi_raw_data[csi_vaid_subcarrier_index[i] * 2],
                                    csi_raw_data[csi_vaid_subcarrier_index[i] * 2 - 1])
        # print(complex(csi_raw_data[csi_vaid_subcarrier_index[i] * 2],
        #                             csi_raw_data[csi_vaid_subcarrier_index[i] * 2 - 1]))
    # 在这里可以对csi_data_array进行进一步处理
    # 例如，分析数据，存储到数据库，或者更新显示等
